In [16]:
"""
1호선 서울교통공사 열차운행시각표 데이터 분석
- 시간 데이터 변환 및 정렬
- 역별/시간대별 운행 빈도 분석
- 배차간격 분석
"""
import pandas as pd
import numpy as np
from datetime import timedelta

In [17]:

# ============================================================
# 1. 데이터 로드 및 전처리
# ============================================================
df = pd.read_csv(
    r'data\(1호선)서울교통공사_서울 도시철도 열차운행시각표_20250704.csv',
    encoding='euc-kr'
)

# 시간 데이터 변환 (object → datetime)
df['열차도착시간'] = pd.to_datetime(df['열차도착시간'], format='%H:%M:%S', errors='coerce')
df['열차출발시간'] = pd.to_datetime(df['열차출발시간'], format='%H:%M:%S', errors='coerce')

# 시간대 추출
df['시간대'] = df['열차출발시간'].dt.hour
df['분'] = df['열차출발시간'].dt.minute

# 급행/일반 구분 컬럼 추가
df['열차구분'] = df['급행여부'].map({0: '일반', 1: '급행'})

print("="*70)
print("📊 1호선 열차운행시각표 데이터 분석")
print("="*70)
print(f"데이터 형태: {df.shape}")
print(f"분석 기간: 주중/주말/공휴일 포함")
print(f"총 역 개수: {df['역사명'].nunique()}개")
print(f"총 열차 코드: {df['열차코드'].nunique()}개")

📊 1호선 열차운행시각표 데이터 분석
데이터 형태: (88431, 15)
분석 기간: 주중/주말/공휴일 포함
총 역 개수: 102개
총 열차 코드: 1019개


In [18]:
# ============================================================
# 2. 기본 통계 분석
# ============================================================
print("\n" + "="*70)
print("📈 기본 통계")
print("="*70)

print("\n1️⃣ 주중/주말/공휴일별 운행 횟수")
print(df['주중주말'].value_counts())

print("\n2️⃣ 방향별 운행 횟수")
print(df['방향'].value_counts())

print("\n3️⃣ 급행/일반 운행 횟수")
print(df['열차구분'].value_counts())

print("\n4️⃣ 운행 횟수가 많은 역 TOP 10")
print(df['역사명'].value_counts().head(10))


📈 기본 통계

1️⃣ 주중/주말/공휴일별 운행 횟수
주중주말
DAY    31303
END    28564
SAT    28564
Name: count, dtype: int64

2️⃣ 방향별 운행 횟수
방향
UP      44234
DOWN    44197
Name: count, dtype: int64

3️⃣ 급행/일반 운행 횟수
열차구분
일반    68791
급행    19640
Name: count, dtype: int64

4️⃣ 운행 횟수가 많은 역 TOP 10
역사명
구로     2151
신도림    1996
영등포    1996
노량진    1872
대방     1872
용산     1872
신길     1872
서울역    1411
동대문    1406
종각     1406
Name: count, dtype: int64


In [19]:


# ============================================================
# 3. 시간대별 분석
# ============================================================
print("\n" + "="*70)
print("⏰ 시간대별 분석")
print("="*70)

print("\n시간대별 운행 빈도")
time_freq = df['시간대'].value_counts().sort_index()
for hour, count in time_freq.items():
    if pd.notna(hour):
        bar = '█' * (count // 500)
        print(f"{int(hour):2d}시: {count:5d}회 {bar}")

# 출퇴근 시간대 분석
morning_rush = df[(df['시간대'] >= 7) & (df['시간대'] <= 9)]
evening_rush = df[(df['시간대'] >= 18) & (df['시간대'] <= 20)]

print(f"\n출근시간대(7~9시): {len(morning_rush):,}회")
print(f"퇴근시간대(18~20시): {len(evening_rush):,}회")


⏰ 시간대별 분석

시간대별 운행 빈도
 0시:    18회 
 5시:  2285회 ████
 6시:  4349회 ████████
 7시:  5073회 ██████████
 8시:  5119회 ██████████
 9시:  4698회 █████████
10시:  4636회 █████████
11시:  4676회 █████████
12시:  4522회 █████████
13시:  4495회 ████████
14시:  4498회 ████████
15시:  4645회 █████████
16시:  4850회 █████████
17시:  4951회 █████████
18시:  5070회 ██████████
19시:  5079회 ██████████
20시:  4812회 █████████
21시:  4506회 █████████
22시:  4134회 ████████
23시:  3081회 ██████

출근시간대(7~9시): 14,890회
퇴근시간대(18~20시): 14,961회


In [20]:

# ============================================================
# 4. 역별 배차간격 분석
# ============================================================
print("\n" + "="*70)
print("🚇 역별 배차간격 분석 (주중 오전 출근시간 7~9시 기준)")
print("="*70)

def analyze_dispatch_interval(station_name, direction='DOWN', time_start=7, time_end=9, weekday='DAY'):
    """특정 역의 배차간격 분석"""
    # 필터링
    condition = (
        (df['역사명'] == station_name) &
        (df['방향'] == direction) &
        (df['주중주말'] == weekday) &
        (df['시간대'] >= time_start) &
        (df['시간대'] < time_end) &
        (df['열차출발시간'].notna())
    )
    
    station_df = df[condition].copy()
    station_df = station_df.sort_values('열차출발시간')
    
    if len(station_df) < 2:
        return None
    
    # 배차간격 계산
    station_df['배차간격_초'] = station_df['열차출발시간'].diff().dt.total_seconds()
    
    # 통계
    result = {
        '역명': station_name,
        '운행횟수': len(station_df),
        '평균배차간격_분': station_df['배차간격_초'].mean() / 60,
        '최소배차간격_분': station_df['배차간격_초'].min() / 60,
        '최대배차간격_분': station_df['배차간격_초'].max() / 60,
        '표준편차_분': station_df['배차간격_초'].std() / 60
    }
    
    return result

# 주요 역 분석
major_stations = ['서울역', '용산', '영등포', '구로', '신도림', '노량진', '종각', '동대문']
dispatch_results = []

for station in major_stations:
    result = analyze_dispatch_interval(station)
    if result:
        dispatch_results.append(result)

dispatch_df = pd.DataFrame(dispatch_results)
print("\n주요 역 배차간격 분석 결과 (DOWN 방향)")
print(dispatch_df.to_string(index=False))


🚇 역별 배차간격 분석 (주중 오전 출근시간 7~9시 기준)

주요 역 배차간격 분석 결과 (DOWN 방향)
 역명  운행횟수  평균배차간격_분  최소배차간격_분  최대배차간격_분   표준편차_분
서울역    32  3.709677       3.0       5.0 0.588419
 용산    48  2.446809       0.0       4.5 1.307312
영등포    50  2.387755       0.0       4.5 1.339563
 구로    50  2.357143       0.0       7.0 1.506928
신도림    51  2.370000       0.0       4.5 1.331755
노량진    49  2.406250       0.0       4.5 1.303254
 종각    32  3.725806       3.0       5.0 0.616964
동대문    32  3.741935       3.0       5.0 0.630753


In [21]:
# ============================================================
# 5. 급행/일반 열차 운행 패턴 분석
# ============================================================
print("\n" + "="*70)
print("🚄 급행/일반 열차 운행 패턴")
print("="*70)

print("\n시간대별 급행/일반 비율")
for hour in range(5, 24):
    hour_df = df[df['시간대'] == hour]
    if len(hour_df) > 0:
        express_count = len(hour_df[hour_df['급행여부'] == 1])
        total_count = len(hour_df)
        express_ratio = express_count / total_count * 100
        
        bar = '█' * int(express_ratio / 5)
        print(f"{hour:2d}시: 급행 {express_count:4d}회 ({express_ratio:5.1f}%) {bar}")


🚄 급행/일반 열차 운행 패턴

시간대별 급행/일반 비율
 5시: 급행  218회 (  9.5%) █
 6시: 급행  741회 ( 17.0%) ███
 7시: 급행 1044회 ( 20.6%) ████
 8시: 급행 1219회 ( 23.8%) ████
 9시: 급행 1169회 ( 24.9%) ████
10시: 급행 1120회 ( 24.2%) ████
11시: 급행 1236회 ( 26.4%) █████
12시: 급행 1213회 ( 26.8%) █████
13시: 급행 1147회 ( 25.5%) █████
14시: 급행 1033회 ( 23.0%) ████
15시: 급행 1211회 ( 26.1%) █████
16시: 급행 1259회 ( 26.0%) █████
17시: 급행 1302회 ( 26.3%) █████
18시: 급행 1183회 ( 23.3%) ████
19시: 급행 1184회 ( 23.3%) ████
20시: 급행 1044회 ( 21.7%) ████
21시: 급행  787회 ( 17.5%) ███
22시: 급행  587회 ( 14.2%) ██
23시: 급행  275회 (  8.9%) █


In [22]:

# ============================================================
# 6. 출발역-도착역 노선 분석
# ============================================================
print("\n" + "="*70)
print("🛤️ 주요 노선 분석 (출발역 → 도착역)")
print("="*70)

route_count = df.groupby(['출발역', '도착역']).size().sort_values(ascending=False).head(10)
print("\n운행 횟수가 많은 노선 TOP 10")
for (start, end), count in route_count.items():
    print(f"{start:6s} → {end:6s}: {count:5d}회")


🛤️ 주요 노선 분석 (출발역 → 도착역)

운행 횟수가 많은 노선 TOP 10
동인천    → 용산    :  5772회
용산     → 동인천   :  5400회
인천     → 양주    :  4730회
양주     → 인천    :  4345회
광운대    → 서동탄   :  4000회
서동탄    → 광운대   :  3760회
신창     → 광운대   :  3398회
동두천    → 인천    :  3294회
인천     → 의정부   :  3276회
천안     → 청량리   :  3244회


In [23]:

# ============================================================
# 7. 주중/주말 비교 분석
# ============================================================
print("\n" + "="*70)
print("📅 주중/주말 비교")
print("="*70)

for period, period_name in [('DAY', '주중'), ('SAT', '토요일'), ('END', '공휴일')]:
    df_period = df[df['주중주말'] == period]
    morning = len(df_period[(df_period['시간대'] >= 7) & (df_period['시간대'] <= 9)])
    evening = len(df_period[(df_period['시간대'] >= 18) & (df_period['시간대'] <= 20)])
    total = len(df_period)
    
    print(f"\n{period_name} ({period})")
    print(f"  총 운행: {total:,}회")
    print(f"  출근시간(7~9시): {morning:,}회 ({morning/total*100:.1f}%)")
    print(f"  퇴근시간(18~20시): {evening:,}회 ({evening/total*100:.1f}%)")


print("\n" + "="*70)
print("✅ 분석 완료!")
print("="*70)


📅 주중/주말 비교

주중 (DAY)
  총 운행: 31,303회
  출근시간(7~9시): 5,898회 (18.8%)
  퇴근시간(18~20시): 5,605회 (17.9%)

토요일 (SAT)
  총 운행: 28,564회
  출근시간(7~9시): 4,496회 (15.7%)
  퇴근시간(18~20시): 4,678회 (16.4%)

공휴일 (END)
  총 운행: 28,564회
  출근시간(7~9시): 4,496회 (15.7%)
  퇴근시간(18~20시): 4,678회 (16.4%)

✅ 분석 완료!
